In [1]:
import pandas as pd
import glob
import os
import xml.etree.ElementTree as ET

# Load Data

In [2]:
# Full bvb export

path = 'data/csv/b3kat_export/'
files = glob.glob(os.path.join(path, '*.csv'))

li = []

for filename in files: 
    frame = pd.read_csv(filename, header=0)
    li.append(frame)

df_full = pd.concat(li, axis=0, ignore_index=True)

# No need for all the columns

df_full= df_full[['id', 'lang', 'authors', 'parentId']]

C:\Users\hanna\AppData\Local\Temp/ipykernel_12332/2627611449.py:9: DtypeWarning: Columns (12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  frame = pd.read_csv(filename, header=0)


In [3]:
# signatures

df_sig = pd.read_csv('data/csv/signatures.csv')

df_sig = df_sig[['sys', 'text', 'text_3', 'text_2', 'text_1']]

In [4]:
# Books available in the library

df_freihand = pd.read_csv('data/Freihand.csv', sep=';',  error_bad_lines=False, header=0, 
                          names = ['bvb', 'title', 'year', 'signature', 'collection', 'inventory_nr', 'inventory_date'])

C:\Users\hanna\AppData\Local\Temp/ipykernel_12332/3202955160.py:3: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df_freihand = pd.read_csv('data/Freihand.csv', sep=';',  error_bad_lines=False, header=0,


In [5]:
print(df_freihand.columns)
print(df_full.columns)
print(df_sig.columns)

Index(['bvb', 'title', 'year', 'signature', 'collection', 'inventory_nr',
       'inventory_date'],
      dtype='object')
Index(['id', 'lang', 'authors', 'parentId'], dtype='object')
Index(['sys', 'text', 'text_3', 'text_2', 'text_1'], dtype='object')


### Merging Signatures and Books

In [6]:
# Extracting the known signature types

generic_pattern = r'^([a-zA-Z]{1,2}) (\d+)'

df_freihand[['signature_l', 'signature_n']] = df_freihand.signature.str.extract(generic_pattern, expand=False)


df_freihand['signature_clean'] = df_freihand[['signature_l', 'signature_n']].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1)

# Merging on the signatures

merged = pd.merge(df_freihand, df_sig, left_on='signature_clean', right_on='sys', how= 'left', indicator=True)
both = merged.loc[merged['_merge'] == 'both']

# recognized signatures

rec_sigs = len(both) / len(df_freihand) * 100
print(f'Books in Freihand matched with signature: {rec_sigs:.1f} %')

# signatures not in the list marked with left only

left_only = merged.loc[merged['_merge'] == 'left_only']

# Keeping this to check which ones I'm still missing

left_only.groupby(left_only.signature[:6]).size().to_csv('data/left_only.csv')


Books in Freihand matched with signature: 39.4 %


In [7]:
# Merging on bvb 

both.drop('_merge' , axis=1, inplace=True)
merged = pd.merge(both, df_full, left_on='bvb', right_on='id', how= 'left', indicator=True)

merged.drop_duplicates(inplace=True)

C:\Users\hanna\AppData\Local\Temp/ipykernel_12332/4264963732.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  both.drop('_merge' , axis=1, inplace=True)


In [8]:
merged

,bvb,title,year,signature,collection,inventory_nr,inventory_date,signature_l,signature_n,signature_clean,sys,text,text_3,text_2,text_1,id,lang,authors,parentId,_merge
0,BV000001769,Sozialgeschichte der mittelalterlichen Kunst,1957,Mk 1020-5571,BIB,2019.4415,20190926,Mk,1020,Mk 1020,Mk 1020,interkontinental und Europa allgemein. Mittela...,Australien und Südsee (Ozeanien),Geschichte der Kunst (allgemein oder vermischt...,Kunst allgemein (ohne Italien speziell),NaN,NaN,NaN,NaN,left_only
1,BV001953292,Sozialgeschichte der mittelalterlichen Kunst,1957,Mk 1020-5571,BIB,2019.4415,20190926,Mk,1020,Mk 1020,Mk 1020,interkontinental und Europa allgemein. Mittela...,Australien und Südsee (Ozeanien),Geschichte der Kunst (allgemein oder vermischt...,Kunst allgemein (ohne Italien speziell),BV001953292,ger,"Hauser, Arnold",BV000001769,both
2,BV005113587,Gotische Bildwerke der deutschen Schweiz,1930,Oe 1090-5300,BIB,NaN,0,Oe,1090,Oe 1090,Oe 1090,Schweiz und Liechtenstein,NaN,Geschichte der Plastik (außer Elfenbeinplastik...,Plastik (ohne Italien speziell),BV005113587,ger,"Baier-Futterer, Ilse",NaN,both
3,BV012157780,Das Photographische,1998,Rv 300-5901/a,BIB,2015.386,20150128,Rv,300,Rv 300,Rv 300,Geschichte der Photographie,Geschichte der modernen Reproduktionsverfahren,Moderne Reproduktionsverfahren (nach 1830),Graphik (ohne Italien speziell),BV012157780,ger,"Krauss, Rosalind E.",NaN,both
4,BV002555116,Antikenstudium und Antikenkopie,1958,Mk 20-5530/a,BIB,NaN,0,Mk,20,Mk 20,Mk 20,interkontinental,Australien und Südsee (Ozeanien),Geschichte der Kunst (allgemein oder vermischt...,Kunst allgemein (ohne Italien speziell),NaN,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129519,BV000613746,Roma e le province,1985,De 100-5380/17(1,BIB,1985.2533,20110618,De,100,De 100,De 100,zeitlich durchgehend,sonstige Quellenpublikationen,Geschichte und Kulturgeschichte,Topographie Rom,NaN,NaN,NaN,NaN,left_only
129520,BV000723788,Roma e le province,1985,De 100-5380/17(2,BIB,1985.2533,20110618,De,100,De 100,De 100,zeitlich durchgehend,sonstige Quellenpublikationen,Geschichte und Kulturgeschichte,Topographie Rom,NaN,NaN,NaN,NaN,left_only
129521,BV000723785,Roma e le province,1985,De 100-5380/17(2,BIB,1985.2533,20110618,De,100,De 100,De 100,zeitlich durchgehend,sonstige Quellenpublikationen,Geschichte und Kulturgeschichte,Topographie Rom,NaN,NaN,NaN,NaN,left_only
129523,BV000613746,Roma e le province,1985,De 100-5380/17(2,BIB,1985.2533,20110618,De,100,De 100,De 100,zeitlich durchgehend,sonstige Quellenpublikationen,Geschichte und Kulturgeschichte,Topographie Rom,NaN,NaN,NaN,NaN,left_only


In [9]:
len(merged)

112281

# Export

In [10]:
text_cols = ['text', 'text_1', 'text_2', 'text_3']
for col in text_cols: 
    merged[col] = merged[col].str.replace(' ', '_')

In [11]:
merged

,bvb,title,year,signature,collection,inventory_nr,inventory_date,signature_l,signature_n,signature_clean,sys,text,text_3,text_2,text_1,id,lang,authors,parentId,_merge
0,BV000001769,Sozialgeschichte der mittelalterlichen Kunst,1957,Mk 1020-5571,BIB,2019.4415,20190926,Mk,1020,Mk 1020,Mk 1020,interkontinental_und_Europa_allgemein._Mittela...,Australien_und_Südsee_(Ozeanien),Geschichte_der_Kunst_(allgemein_oder_vermischt...,Kunst_allgemein_(ohne_Italien_speziell),NaN,NaN,NaN,NaN,left_only
1,BV001953292,Sozialgeschichte der mittelalterlichen Kunst,1957,Mk 1020-5571,BIB,2019.4415,20190926,Mk,1020,Mk 1020,Mk 1020,interkontinental_und_Europa_allgemein._Mittela...,Australien_und_Südsee_(Ozeanien),Geschichte_der_Kunst_(allgemein_oder_vermischt...,Kunst_allgemein_(ohne_Italien_speziell),BV001953292,ger,"Hauser, Arnold",BV000001769,both
2,BV005113587,Gotische Bildwerke der deutschen Schweiz,1930,Oe 1090-5300,BIB,NaN,0,Oe,1090,Oe 1090,Oe 1090,Schweiz_und_Liechtenstein,NaN,Geschichte_der_Plastik_(außer_Elfenbeinplastik...,Plastik_(ohne_Italien_speziell),BV005113587,ger,"Baier-Futterer, Ilse",NaN,both
3,BV012157780,Das Photographische,1998,Rv 300-5901/a,BIB,2015.386,20150128,Rv,300,Rv 300,Rv 300,Geschichte_der_Photographie,Geschichte_der_modernen_Reproduktionsverfahren,Moderne_Reproduktionsverfahren_(nach_1830),Graphik_(ohne_Italien_speziell),BV012157780,ger,"Krauss, Rosalind E.",NaN,both
4,BV002555116,Antikenstudium und Antikenkopie,1958,Mk 20-5530/a,BIB,NaN,0,Mk,20,Mk 20,Mk 20,interkontinental,Australien_und_Südsee_(Ozeanien),Geschichte_der_Kunst_(allgemein_oder_vermischt...,Kunst_allgemein_(ohne_Italien_speziell),NaN,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129519,BV000613746,Roma e le province,1985,De 100-5380/17(1,BIB,1985.2533,20110618,De,100,De 100,De 100,zeitlich_durchgehend,sonstige_Quellenpublikationen_,Geschichte_und_Kulturgeschichte,Topographie_Rom,NaN,NaN,NaN,NaN,left_only
129520,BV000723788,Roma e le province,1985,De 100-5380/17(2,BIB,1985.2533,20110618,De,100,De 100,De 100,zeitlich_durchgehend,sonstige_Quellenpublikationen_,Geschichte_und_Kulturgeschichte,Topographie_Rom,NaN,NaN,NaN,NaN,left_only
129521,BV000723785,Roma e le province,1985,De 100-5380/17(2,BIB,1985.2533,20110618,De,100,De 100,De 100,zeitlich_durchgehend,sonstige_Quellenpublikationen_,Geschichte_und_Kulturgeschichte,Topographie_Rom,NaN,NaN,NaN,NaN,left_only
129523,BV000613746,Roma e le province,1985,De 100-5380/17(2,BIB,1985.2533,20110618,De,100,De 100,De 100,zeitlich_durchgehend,sonstige_Quellenpublikationen_,Geschichte_und_Kulturgeschichte,Topographie_Rom,NaN,NaN,NaN,NaN,left_only


In [12]:
# Export

export_columns = ['bvb', 'title', 'signature', 'inventory_nr','inventory_date', 'lang', 'year', 'sys', 'text', 'text_1', 'text_2', 'text_3']

merged[export_columns].to_csv('data/csv/freihand_signatures.csv', index=False)